# ConvNet for MNIST Classification

This example creates a Convolutional Neural Network to classify handwritten digits in the MNIST dataset. ConvNets are a powerful network architecture to create features from images. Below, we will define several network architectures and examine their performance on this well-known dataset. The deep ConvNet produced an error of 0.55% .

This exercise is based on a [Machine Learning Mastery tutorial](http://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras). A good resource for ConvNets is the [CS231n pages on GitHub](http://cs231n.github.io/convolutional-networks).

In [14]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

K.set_image_dim_ordering('th')


# fix random seed for reproducibility
seed = 43
numpy.random.seed(seed)

# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

def baseline_model():
	model = Sequential()
	model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(1, 28, 28), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

def larger_model():
	model = Sequential()
    # create 30 features with a 5x5 kernel
	model.add(Convolution2D(50, 5, 5, border_mode='valid', input_shape=(1, 28, 28), activation='relu'))
    # reduce the spatial size of our first conv layer, reducing the number of weights and thereby overfitting
    # generally, strides of > 2 are too destructive
	model.add(MaxPooling2D(pool_size=(2, 2))) # takes a default stride size of 2, reducing input to 1x14x14
	model.add(Convolution2D(15, 3, 3, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

def deep_model():
    # based on the pattern: INPUT -> [CONV -> RELU -> CONV -> RELU -> POOL]*3 -> [FC -> RELU]*2 -> FC
    # from the CS231n treatment of ConvNet architectures.
	model = Sequential()
	model.add(Convolution2D(40, 3, 3, border_mode='valid', input_shape=(1, 28, 28), activation='relu'))
	model.add(Convolution2D(40, 3, 3, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(20, 3, 3, activation='relu'))
	model.add(Convolution2D(20, 3, 3, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# build the model
model = deep_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Deep Error: %.2f%%" % (100-scores[1]*100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
26s - loss: 0.3019 - acc: 0.9080 - val_loss: 0.0691 - val_acc: 0.9783
Epoch 2/10
25s - loss: 0.0708 - acc: 0.9780 - val_loss: 0.0522 - val_acc: 0.9829
Epoch 3/10
25s - loss: 0.0474 - acc: 0.9857 - val_loss: 0.0456 - val_acc: 0.9851
Epoch 4/10
25s - loss: 0.0384 - acc: 0.9882 - val_loss: 0.0319 - val_acc: 0.9896
Epoch 5/10
25s - loss: 0.0315 - acc: 0.9901 - val_loss: 0.0311 - val_acc: 0.9891
Epoch 6/10
25s - loss: 0.0252 - acc: 0.9917 - val_loss: 0.0326 - val_acc: 0.9899
Epoch 7/10
25s - loss: 0.0222 - acc: 0.9930 - val_loss: 0.0296 - val_acc: 0.9908
Epoch 8/10
25s - loss: 0.0197 - acc: 0.9940 - val_loss: 0.0294 - val_acc: 0.9910
Epoch 9/10
25s - loss: 0.0179 - acc: 0.9945 - val_loss: 0.0369 - val_acc: 0.9896
Epoch 10/10
25s - loss: 0.0165 - acc: 0.9949 - val_loss: 0.0310 - val_acc: 0.9910
Deep Error: 0.90%
